In [5]:
import os
from os import listdir
from os.path import isfile, join
from pathlib import Path
import re

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle


import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import random
%matplotlib inline
SEED = 1105


In [9]:
print(tf.version.VERSION)

2.2.0


In [10]:
HYPER_PARAMETERS = {}
# Position
POSITION = 'Front pocket (trouser)_Left'
POSITION = 'Wrist_Right'
HYPER_PARAMETERS["pos"] = POSITION
# Window size or the sequence length
N_STEPS = 30
HYPER_PARAMETERS["n_steps"] = N_STEPS
# test ratio size, 0.2 is 20%
TEST_SIZE = 0.2
HYPER_PARAMETERS["test_size"] = TEST_SIZE
# date now
date_now = time.strftime("%Y-%m-%d")
HYPER_PARAMETERS["date"] = date_now



### model parameters
N_LAYERS = 2
HYPER_PARAMETERS["n_layers"] = N_LAYERS
# LSTM cell
CELL = LSTM
HYPER_PARAMETERS["cell"] = CELL
# 256 LSTM neurons
UNITS = 64
HYPER_PARAMETERS["units"] = UNITS
# 40% dropout
DROPOUT = 0.4
HYPER_PARAMETERS["dropout"] = DROPOUT
# whether to use bidirectional RNNs
BIDIRECTIONAL = False
HYPER_PARAMETERS["bidirectional"] = BIDIRECTIONAL


### training parameters
# mean absolute error loss
# LOSS = "mae"
# huber loss
LOSS = "huber_loss"
LOSS = "binary_crossentropy"
HYPER_PARAMETERS["loss"] = LOSS
OPTIMIZER = "adam"
HYPER_PARAMETERS["optimizer"] = OPTIMIZER
BATCH_SIZE = 1024
HYPER_PARAMETERS["batch_size"] = BATCH_SIZE
EPOCHS = 100
HYPER_PARAMETERS["epochs"] = EPOCHS

POSSIBLE_ACTIVITIES = ['Climbing stairs',
                       'Descending stairs',
                       'Desk work',
                       'Lying down left', 
                       'Lying down right',
                       'Lying down on back', 
                       'Lying down on stomach', 
                       'Running', 
                       'Sitting bent backward', 
                       'Sitting bent forward', 
                       'Sitting', 
                       'Standing', 
                       'Walking at normal speed',
                       'Movement']    

In [11]:
def create_model(sequence_length, output_size, units=256, cell=LSTM, n_layers=2, dropout=0.3,
                loss="mean_absolute_error", optimizer="rmsprop", bidirectional=False):
    model = Sequential()
    model.add(Dense(sequence_length))
    model.add(Dense(sequence_length))
    for i in range(n_layers):
        if i == 0:
            # first layer
            if bidirectional:
                model.add(Bidirectional(cell(units, return_sequences=True), input_shape=(sequence_length, 3)))
            else:
                model.add(cell(units, return_sequences=True, input_shape=(sequence_length, 3)))
        elif i == n_layers - 1:
            # last layer
            if bidirectional:
                model.add(Bidirectional(cell(units, return_sequences=False)))
            else:
                model.add(cell(units, return_sequences=False))
        else:
            # hidden layers
            if bidirectional:
                model.add(Bidirectional(cell(units, return_sequences=True)))
            else:
                model.add(cell(units, return_sequences=True))
        # add dropout after each layer
        model.add(Dropout(dropout))
    model.add(Dense(output_size))
    model.add(Dense(output_size))
    model.add(Dense(output_size, activation="sigmoid"))
    model.compile(loss=loss, 
                  metrics=["mean_absolute_error", tf.keras.metrics.CategoricalAccuracy()], 
                  optimizer=optimizer,)
    return model
def create_models(sequence_length, possible_activities, units=256, cell=LSTM, n_layers=2, dropout=0.3,
                loss="mean_absolute_error", optimizer="rmsprop", bidirectional=False):
    
    models = {}
    for activity in possible_activities:
        models[activity] = create_model(sequence_length = sequence_length,
                                        output_size = 2, 
                                        loss = loss, 
                                        units = units, 
                                        cell = cell, 
                                        n_layers = n_layers,
                                        dropout = dropout, 
                                        optimizer = optimizer, 
                                        bidirectional = bidirectional)
    return models

In [25]:
data = np.ones([1, 30, 3])
def load_models(files):
    
    
    models = create_models(sequence_length=N_STEPS,
                       possible_activities = POSSIBLE_ACTIVITIES, 
                       loss=LOSS,
                       units=UNITS,
                       cell=CELL,
                       n_layers=N_LAYERS,
                       dropout=DROPOUT, 
                       optimizer=OPTIMIZER, 
                       bidirectional=BIDIRECTIONAL)
    
    for activity, file in zip(POSSIBLE_ACTIVITIES, files):
        model = models[activity]
        model.predict(data)
        model.load_weights(file)
        models[activity] = model
        
        
    return models
    return models
def convert_models(models,position):
    for activity in POSSIBLE_ACTIVITIES:
        converter = tf.lite.TFLiteConverter.from_keras_model(models[activity])
        tflite_model = converter.convert()
        open('converted_models/'+position+'_'+activity+".tflite", "wb").write(tflite_model)
        

In [26]:
files

['models/2020-11-23_Wrist_Right_Desk work_binary_crossentropy_adam_seq-30_layers-2_units-64.h5',
 'models/2020-11-23_Wrist_Right_Standing_binary_crossentropy_adam_seq-30_layers-2_units-64.h5',
 'models/2020-11-23_Wrist_Right_Descending stairs_binary_crossentropy_adam_seq-30_layers-2_units-64.h5',
 'models/2020-11-23_Wrist_Right_Lying down on stomach_binary_crossentropy_adam_seq-30_layers-2_units-64.h5',
 'models/2020-11-23_Wrist_Right_Movement_binary_crossentropy_adam_seq-30_layers-2_units-64.h5',
 'models/2020-11-23_Wrist_Right_Lying down left_binary_crossentropy_adam_seq-30_layers-2_units-64.h5',
 'models/2020-11-23_Wrist_Right_Lying down on back_binary_crossentropy_adam_seq-30_layers-2_units-64.h5',
 'models/2020-11-23_Wrist_Right_Running_binary_crossentropy_adam_seq-30_layers-2_units-64.h5',
 'models/2020-11-23_Wrist_Right_Sitting bent backward_binary_crossentropy_adam_seq-30_layers-2_units-64.h5',
 'models/2020-11-23_Wrist_Right_Lying down right_binary_crossentropy_adam_seq-30_lay

In [27]:
import tensorflow.keras.models as md

target_folder ='./models'
dir = Path(target_folder)
files = [dir/f for f in listdir(target_folder) if isfile(join(target_folder, f)) and os.path.splitext(f)[1] == '.h5']
files = [str(f) for f in files]
models = load_models(files)
convert_models(models,POSITION)

In [34]:
models = create_models(sequence_length=N_STEPS,
                       possible_activities = POSSIBLE_ACTIVITIES, 
                       loss=LOSS,
                       units=UNITS,
                       cell=CELL,
                       n_layers=N_LAYERS,
                       dropout=DROPOUT, 
                       optimizer=OPTIMIZER, 
                       bidirectional=BIDIRECTIONAL)
data = np.ones([1, 30, 3])


models['Climbing stairs'].predict(data).shape


(1, 2)